# LeNet-5

[LeNet-5](http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf)是由Yann LeCun在上世纪90年代实现的.

其是一种典型的用来识别数字的卷积网络,当年美国大多数银行就是用它来识别支票上面的手写数字的. 效果图如下所示:

<p><img src="http://yann.lecun.com/exdb/lenet/gifs/asamples.gif" nosave="" height="200" width="320">&nbsp;
<br><img src="http://yann.lecun.com/exdb/lenet/gifs/legend-bottom.gif" nosave="" height="80" width="320">&nbsp;
</p>

**Ps:**

一下的阐述是针对原文论而言,虽然目前CNN中已经不再采取其中的某些思想,但是针对于原文而言,我们还是需要作为一个了解.

原论文结构如下:

![](../../../picture/177.png)

以Ng的模型显示如下:

![](../../../picture/178.png)

**LeNet-5**:不包括Input layers一共有7层,需要注意的是,输入图像为$(32,32)$,是超过Minist数据集$(28,28)$的,这样做是为了希望潜在的明显特征如数字断点,角点能够出现在第一层感受野的中心点.而且我们需要明确一点:每一层有多个Feature Map,也就是多个特征图,每个Feature Map通过一种卷积核提取输入一种特征,然后每个Feature Map有多个神经元.

下面我们来理解下网络的每一层:

**C1:**该层为卷积层,卷积运算的一个重要特点是通过卷积运算,可以使原信号增强并且降低噪声.由6个特征图构成,每一个特征图与输入的$(5,5)$卷积核领域相连接.Feature Map的大小为$(28,28)$,这样能够防止连接掉到边界之外,也就是说正好能感受野滑完整,这样以便于Bp时候的计算.C1有156个可训练参数(每个滤波器$5*5=25$个unit参数和一个bias参数,一共6个滤波器,共$(5*5+1)*6=156$个参数),共$156*(28*28)=122,304$个连接.

**S2(Subsamping):**该层为$Average\;Pool$,下采样的好处是利用图像局部相关性原理,对图像进行子抽样,可以达到减少数据的同时保留有用信息,不过现在一般很少使用$AP$,转而使用$MP$.有6个$(14,14)$的特征图中每一个单元与C1中相对应的感受野$(2,2)$领域链接.结果通过**sigmoid**函数计算.使用sigmoid的好处是可以控制训练系数和偏置的非线性程度.如果系数$W$较小,那么运算近似是线性运算,[亚采样](https://zhuanlan.zhihu.com/p/22445302)相当于模糊图像.如果系数比较大,根据偏置的大小亚采样可以被看成是有噪声的“或”运算或者有噪声的“与”运算.每个单元的$2*2$感受野并不重叠,因此S2层有12个可训练参数和5880个连接:

- 12个可训练参数:6个$2*2$的感受野,每个有一个参数,加上一个偏置,也就是$(1+1)*6=12$个.
- 5880个连接:对于S2层上的每一个图的每一个点,连接数是$(2*2+1)=5$,共$14*14*6*(2*2+1)=5880$

**Ps:**
> 一般对于现代CNN而言,我们已经很少在使用sigmoid函数了,且以现代的观点而言,我们的Pool层是没有训练参数的.

**C3:**该层也是一个卷积层,它同样通过$(5,5)$的卷积核去卷积$S2$,得到的特征图为$(10,10)$也就是$(10\times10)$个神经元,但是它有16种不同的卷积核,所以就存在16个特征图.C3中的每个特征图是连接到S2中的所有6个或者几个特征图的,表示本层的特征图是上一层提取到的特征图的不同组合(这个做法也并不是唯一的),这里原文作者举例一个例子:

例如,存在的一个方式是:C3的前6个特征图以S2中3个相邻的特征图子集为输入.接下来6个特征图以S2中4个相邻特征图子集为输入.然后的3个以不相邻的4个特征图子集为输入.最后一个将S2中所有特征图为输入.这样C3层有1516个可训练参数和151600个连接:
> ![](../../../picture/179.png)

- 1516个可训练参数:共有16个特征图,那么每个对应的特征图之和$6*(3*25+1) + 6*(4*25+1) + 3*(4*25+1)+ 1*(6*25+1)=1516$

- 151600个连接: $(10*10)$的特征图中每一个都与上述参数连接,则总连接为$(10*10*1516)=151600$

**S4:**该层是一个下采样层($AP$),由16个$5*5$大小的特征图构成.特征图中的每个单元与C3中相应特征图的$2*2$邻域相连接,跟C1和S2之间的连接一样.S4层有32个可训练参数(每个特征图1个因子和一个偏置)和2000个连接.

- 32个可训练参数:16个$2*2$的小方框,每个有一个参数,加上一个偏置,也就是$(1+1)*16=32$.

- 2000个连接:对于S4层的每一个图的每一个点,连接数是$(2*2+1)=5$,总共是$5*5*16*(2*2+1)=2000$

**C5:**该层是一个卷积层,有120个特征图.每个单元与S4层的全部16个单元的$5*5$邻域相连.由于S4层特征图的大小也为$5*5$也就是使用同等大小的卷积核,故C5特征图的大小为$1*1$:这构成了S4和C5之间的全连接.之所以仍将C5标示为卷积层而非全相联层,是因为如果LeNet-5的输入变大,而其他的保持不变,那么此时特征图的维数就会比$1*1$大.C5层有48120个可训练连接.

**Ps:**

- 这里所说的就是全卷积层,我们在CNN(Theory)中提到过,在之后我们会再来了解下全卷积层代替全连接层.

- 48120个可训练连接:120*(5*5*16+1)=48120


**F6:**该层有84个单元(之所以选这个数字的原因来自于输出层的设计),与C5层全相连.有10164个可训练参数.如同经典神经网络,F6层计算输入向量和权重向量之间的点积,再加上一个偏置,然后将其传递给sigmoid函数产生单元**i**的一个状态.

- 10164个可训练连接:$84*(120+1)=10164$

**Ps:**
>此处为论文对F6层和输出层的解释:
>输出层由欧式径向基函数[(Euclidean Radial Basis Function)](https://zh.wikipedia.org/wiki/%E5%BE%84%E5%90%91%E5%9F%BA%E5%87%BD%E6%95%B0%E6%A0%B8)单元组成,每类一个单元,每个有84个输入.换句话说,每个输出RBF单元计算输入向量和参数向量之间的欧式距离.输入离参数向量越远,RBF输出的越大.一个RBF输出可以被理解为衡量输入模式和与RBF相关联类的一个模型的匹配程度的惩罚项.用概率术语来说,RBF输出可以被理解为F6层配置空间的高斯分布的负log-likelihood.给定一个输入模式,损失函数应能使得F6的配置与RBF参数向量(即模式的期望分类)足够接近.这些单元的参数是人工选取并保持固定的(至少初始时候如此).这些参数向量的成分被设为-1或1.虽然这些参数可以以-1和1等概率的方式任选,或者构成一个纠错码,但是被设计成一个相应字符类的$7*12$大小(即84)的格式化图片.这种表示对识别单独的数字不是很有用,但是对识别可打印ASCII集中的字符串很有用.使用这种分布编码而非更常用的"1 of N"编码用于产生输出的另一个原因是,当类别比较大的时候,非分布编码的效果比较差.原因是大多数时间非分布编码的输出必须为0.这使得用sigmoid单元很难实现.另一个原因是分类器不仅用于识别字母,也用于拒绝非字母.使用分布编码的RBF更适合该目标.因为与sigmoid不同,他们在输入空间的较好限制的区域内兴奋,而非典型模式更容易落到外边.
RBF参数向量起着F6层目标向量的角色.需要指出这些向量的成分是+1或-1,这正好在F6 sigmoid的范围内,因此可以防止sigmoid函数饱和.实际上,+1和-1是sigmoid函数的最大弯曲的点处.这使得F6单元运行在最大非线性范围内.必须避免sigmoid函数的饱和,因为这将会导致损失函数较慢的收敛和病态问题.

> RBF核保有的常向量对应的bitmap字符其是$(7*12)$:

> ![](https://www.charleychai.com/images/blogs/2018/lenet_rbf.png)

**output:** The output layer is composed of Euclidean Radial Basis Function units (RBF), one for each class with 84 inputs each.The outputs of each RBF unit $y_i$:

$y_i=\underset{j}{\sum}(x_j-w_{ij})^2$

到此LeNet-5的网络已经解释清楚,但是如果想要源生完整实现LeNet-5还是比较困难的,对于$S2\rightarrow C3$就是一个比较难的过程,还有F6层(RBF层)也是很难复现的,在网上也很难找到好的复现代码,这里只找到一篇关于[$S2\rightarrow C3$](https://github.com/tiny-dnn/tiny-dnn/tree/master/examples/mnist)复现的方法.目前先使用当前的$Softmax$实现,以后有机会再来尝试源生实现.

**LeNet-5特点:**

(1) 每个卷积层包括三个部分:卷积,池化,非线性激活函数

(2) 使用卷积提取空间特征

(3) 降采样(Subsample)的平均池化层($AP$)

(4) 使用$Tanh$或者$Sigmoid$激活函数[MLP](https://blog.csdn.net/u012162613/article/details/43221829)作为最后的分类器

(5) 层与层之间稀疏连接,减少计算复杂度.

**Summary:**

1) LeNet-5网络的参数大约是60K个,而且图像的高度和宽度都在缩小$(n_H,n_W)$,而通道在增大$(n_C)$.

2) 实际上现在这种CONV->POOL->CONV->POOL-FC-FC-OUTPUT这种模式还是经常被用到.

**Reference:**

[1] [LeNet-5卷积参数](https://blog.csdn.net/qiaofangjie/article/details/16826849)

[2] [LeNet-5, convolutional neural networks](http://yann.lecun.com/exdb/lenet/index.html)

[3] [卷积神经网络的经典网络](https://www.cnblogs.com/guoyaohua/p/8534077.html)

[4] [LeNet-5的原理](https://zhuanlan.zhihu.com/p/27256298)